In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=060&bs=040&ra=027&cb=0.0&ct=9999999&et=5&cn=10&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&rn=2415'
res = requests.get(url)

In [3]:
res

<Response [200]>

In [4]:
res.text

'<!DOCTYPE html>\r\n<!--[if lt IE 7]><html lang="ja" class="ie"><![endif]-->\r\n<!--[if IE 7]><html lang="ja" class="ie"><![endif]-->\r\n<!--[if IE 8]><html lang="ja" class="ie"><![endif]-->\r\n<!--[if gt IE 8]><!--><html lang="ja"><!--<![endif]-->\r\n<!-- FR301FC0012 START -->\r\n<head>\r\n<meta charset="utf-8" />\r\n<!-- seo.headerTag Start -->\r\n\r\n<title>【SUUMO】 ＪＲ大阪環状線の賃貸住宅・賃貸マンション／アパートのお部屋探し物件情報</title>\r\n\r\n<meta name="keywords" content=\'ＪＲ大阪環状線,賃貸,賃貸マンション,アパート,賃貸アパート,賃貸情報,部屋探し,賃貸住宅,物件,賃貸物件,住まい,SUUMO,スーモ\' />\r\n\r\n<meta name="description" content=\'ＪＲ大阪環状線の賃貸マンション、アパートのお部屋探し情報一覧です。賃貸マンション・賃貸アパートなど、賃貸住宅のお部屋探しならリクルートの賃貸情報サイトSUUMO(スーモ) 。ＪＲ大阪環状線（大阪府）の豊富な賃貸情報からあなたにピッタリの住まい情報を見つけてください。\' />\r\n\r\n<meta name="robots" content="noindex, follow" />\r\n<link rel="shortcut icon" href="/front/img/favicon.ico" type="image/vnd.microsoft.icon" />\r\n<link rel="icon" href="/front/img/favicon.ico" type="image/vnd.microsoft.icon" />\r\n\r\n<meta name="robots" content="noarchive">\r\n<!-- s

In [5]:
soup = BeautifulSoup(res.text, "html.parser")

# 1~50ページの情報を取得

In [8]:
data = []

url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=060&bs=040&ra=027&cb=0.0&ct=9999999&et=5&cn=10&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&rn=2415'
res = requests.get(url)
soup = BeautifulSoup(res.text,'html.parser')

body = soup.find("body")
pages = body.find_all("div", {'class':'pagination pagination_set-nav'})
pages_text = str(pages)
pages_split = pages_text.split('</a></li>\n</ol>')
num_pages = int(pages_split[0].split('>')[-1])

for i in range(1,num_pages):
    url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=060&bs=040&ra=027&cb=0.0&ct=9999999&et=5&cn=10&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&rn=2415&page={}'.format(i)
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'html.parser')
    bukkens = soup.find_all('div',attrs={'class':'cassetteitem'})

    for bukken in bukkens:
        # 最寄駅
        station_info = bukken.select(".cassetteitem_detail-text")
        railline = station_info[0].text.split('/')[0]
        station = station_info[0].text.split('/')[1].split(' ')[0].replace('駅','')
        # 駅までの所要時間
        how_long = station_info[0].text.split('/')[1].split(' ')[1]
        how_long = int(re.sub(r"\D", "", how_long) )
        # 家賃
        rent_price = bukken.find_all('span',attrs={'class':'cassetteitem_price cassetteitem_price--rent'})[0].text
        rent_price = float(rent_price.replace("万円",""))
        # 間取り
        room_type = bukken.find_all('span',attrs={'class':'cassetteitem_madori'})[0].text
        # 占有面積
        area = bukken.find_all('span',attrs={'class':'cassetteitem_menseki'})[0].text
        area = float(area.replace("m2",""))
        
        datum = {}
        
        datum['路線']=railline
        datum['最寄駅']=station
        datum['駅までの所要時間（分）']=how_long
        datum['家賃（万円）']=rent_price
        datum['間取り']=room_type
        datum['占有面積']=area
        data.append(datum)

In [9]:
data

[{'路線': 'ＪＲ大阪環状線',
  '最寄駅': '今宮',
  '駅までの所要時間（分）': 5,
  '家賃（万円）': 5.3,
  '間取り': '1K',
  '占有面積': 23.8},
 {'路線': '地下鉄御堂筋線',
  '最寄駅': '大国町',
  '駅までの所要時間（分）': 8,
  '家賃（万円）': 5.8,
  '間取り': '1K',
  '占有面積': 21.77},
 {'路線': 'ＪＲ大阪環状線',
  '最寄駅': '天王寺',
  '駅までの所要時間（分）': 5,
  '家賃（万円）': 5.75,
  '間取り': '1K',
  '占有面積': 21.74},
 {'路線': 'ＪＲ大阪環状線',
  '最寄駅': '玉造',
  '駅までの所要時間（分）': 4,
  '家賃（万円）': 6.06,
  '間取り': '1K',
  '占有面積': 23.34},
 {'路線': 'ＪＲ大阪環状線',
  '最寄駅': '福島',
  '駅までの所要時間（分）': 5,
  '家賃（万円）': 7.2,
  '間取り': '1K',
  '占有面積': 25.16},
 {'路線': '地下鉄御堂筋線',
  '最寄駅': '大国町',
  '駅までの所要時間（分）': 11,
  '家賃（万円）': 5.3,
  '間取り': '1K',
  '占有面積': 23.8},
 {'路線': 'ＪＲ大阪環状線',
  '最寄駅': '玉造',
  '駅までの所要時間（分）': 5,
  '家賃（万円）': 14.0,
  '間取り': '1LDK',
  '占有面積': 50.01},
 {'路線': '地下鉄御堂筋線',
  '最寄駅': '大国町',
  '駅までの所要時間（分）': 6,
  '家賃（万円）': 6.08,
  '間取り': '1K',
  '占有面積': 24.48},
 {'路線': '地下鉄長堀鶴見緑地線',
  '最寄駅': '玉造',
  '駅までの所要時間（分）': 1,
  '家賃（万円）': 10.7,
  '間取り': '1LDK',
  '占有面積': 42.91},
 {'路線': '地下鉄御堂筋線',
  '最寄駅': '大国町',
  '駅までの所要時間（分）

In [10]:
import pandas as pd

In [11]:
df = pd.DataFrame(data)
df

,路線,最寄駅,駅までの所要時間（分）,家賃（万円）,間取り,占有面積
0,ＪＲ大阪環状線,今宮,5,5.30,1K,23.80
1,地下鉄御堂筋線,大国町,8,5.80,1K,21.77
2,ＪＲ大阪環状線,天王寺,5,5.75,1K,21.74
3,ＪＲ大阪環状線,玉造,4,6.06,1K,23.34
4,ＪＲ大阪環状線,福島,5,7.20,1K,25.16
...,...,...,...,...,...,...
985,ＪＲ大阪環状線,京橋,2,6.90,2K,25.37
986,地下鉄谷町線,天神橋筋六丁目,2,6.20,1K,22.12
987,ＪＲ大阪環状線,京橋,5,10.50,1LDK,36.95
988,地下鉄堺筋線,扇町,2,8.20,1DK,31.88


In [12]:
df.columns

Index(['路線', '最寄駅', '駅までの所要時間（分）', '家賃（万円）', '間取り', '占有面積'], dtype='object')

In [13]:
df.to_csv('suumo_kanjosen(toho5min_chikunenn10years).csv',index=None)